In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup as bs
from nltk.corpus import stopwords

In [2]:
def process(chk,i):
    pat = re.compile(r'\d+\s+U\.S.\s+\d+', re.MULTILINE|re.UNICODE)
    val = []
    chk = chk.drop_duplicates(subset='html',keep='first')
    not_nan = chk[pd.notnull(chk['html'])]
    for i,c in enumerate(not_nan['html']): 
        try:                                                     
            patmatch = re.findall(pat,c)[0]
            val.append(patmatch)
        except: 
            val.append(None)
            
    not_nan['US_CITE'] = val
    not_nan = not_nan[pd.notnull(not_nan['US_CITE'])]

    gs = pd.read_csv('case_meta_data.csv', encoding = "ISO-8859-1")
    gs['US_CITE'] = gs['usCite']
    #print(gs.head(1))
    #print("")
    #print(not_nan.head(1))
    
    result = pd.merge(not_nan,gs,how='left',on='US_CITE')
    #print(result.head(1))

    result = result.drop(['Unnamed: 0','absolute_url','author','author_str','cluster','date_created','date_modified','date_created','date_modified','download_url','extracted_by_ocr','html_columbia','html_lawbox','html_with_citations','joined_by','local_path','opinions_cited','page_count','per_curiam','plain_text','resource_uri','sha1','type','caseId','usCite'], axis=1)
    
    html_cleaner = lambda x: bs(x).text
    result['html'] = result['html'].apply(html_cleaner)
    # column you are working on
    html_df = result['html']
    stopword_set = set(stopwords.words("english"))
    # convert to lower case and split 
    html_df = html_df.str.lower()
    #print(html_df.head(2))
    splitter = lambda x: x.split()
    html_df.apply(splitter)
    #print(html_df.head(2))
    # remove stopwords
    html_df_list = html_df.tolist()
    html_df_wo_stops = [w for w in html_df_list if not w in stopwords.words("english")]
    #print(html_df_list[:2])
    # keep only words
    html_only_words = [re.sub(r'[^a-zA-Z\s]','', i) for i in html_df_wo_stops]
    #print(html_only_words[:2])
    # join the cleaned words in a list
    html_df = pd.DataFrame({"processed_html": html_only_words})
    html_df.replace("\n", " ").replace('\r', '')
    result['html'] = html_df

    result.to_csv(str(i)+'-proc.csv')

In [3]:
chunk_size = 10**4
i = 0

for chunk in pd.read_csv('scotus_file',chunksize=chunk_size):
    process(chunk, i)
    i += 1

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [ ]:
gs.head(2)

In [ ]:
result.head(2)

In [ ]:
df = pd.read_csv("9471-proc.csv")

In [ ]:
df.info()

In [ ]:
len(df['US_CITE'].unique())

In [ ]:
len((df['html'][pd.isnull(df['US_CITE'])]).unique())

sum(df.US_CITE.isnull())

In [ ]:
df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,20,21,22,24,25,26]], axis=1)
df

In [ ]:
from bs4 import BeautifulSoup as bs
from nltk.corpus import stopwords

html_cleaner = lambda x: bs(x).text
df['html'] = df['html'].apply(html_cleaner)

# column you are working on
html_df = df['html'].head(50)

stopword_set = set(stopwords.words("english"))

# convert to lower case and split 
html_df = html_df.str.lower()
splitter = lambda x: x.split()
html_df.apply(splitter)

# remove stopwords
html_df_list = html_df.tolist()
html_df_wo_stops = [w for w in html_df_list if not w in stopwords.words("english")]

# keep only words
html_only_words = [re.sub(r'[^a-zA-Z]',' ', i) for i in html_df_wo_stops]

# join the cleaned words in a list
html_df = pd.DataFrame({"processed_html": html_only_words})
html_df.replace("\n", " ").replace('\r', '')

df['html'] = html_df

In [ ]:
df1 = df.copy()

In [ ]:
df.append(df1, ignore_index=True)